# Project Demo

This demo is designed to provide some general tips and tricks for the ITDS Fall 2019 project. For full details on the project, please refer to [the project requirements](https://grantmlong.com/teaching/fall2019/project/Project-ITDS-Fall-2019.pdf).

***
This demo uses `scikit-learn`, but by no means are you required to use this or any other particular package.



In [0]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
import seaborn as sns
import io
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [127]:
from google.colab import files
uploaded = files.upload()

Saving school_data.csv to school_data (1).csv


In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)
schools_df = pd.read_csv(io.BytesIO(uploaded['school_data.csv']), index_col=0)

#train_index = train_df.index.copy()#copies the original index
#train_index['addr_zip'] = train_df['addr_zip']


train_df = pd.DataFrame.join(train_df, schools_df)


test_df = pd.merge(test_df, schools_df,left_on='addr_zip', right_on='zipcode')

submit1_df = pd.merge(submit1_df, schools_df,left_on='addr_zip', right_on='zipcode')
submit2_df = pd.merge(submit2_df, schools_df,left_on='addr_zip', right_on='zipcode')

train_df = train_df.drop(train_df[(train_df['size_sqft']>8000) & (train_df['rent']<10000)].index)
train_df = train_df.drop(train_df[(train_df['bathrooms']>=12) & (train_df['rent']<10000)].index)

train_df = train_df.drop(train_df[(train_df['bathrooms']==6) & (train_df['rent']<20000)].index)

train_df = train_df.drop(train_df[(train_df['bedrooms']>=8) & (train_df['rent']<20000)].index)


cf = submit1_df #retaining the dataframe in cf instead incase I want to look at the original without edits
cf.min_to_subway = cf.min_to_subway.fillna(cf.min_to_subway.mean(), axis=0) #Replacing empty values with the mean of the entire column
cf.year_built = cf.year_built.fillna(cf.year_built.median(), axis=0) #Replacing empty values with the median of the entire column
cf.loc[cf['size_sqft']==0,'size_sqft'] = np.nan
cf.size_sqft = cf.size_sqft.fillna(cf.size_sqft.mean(), axis=0)


cf2 = submit2_df #retaining the dataframe in cf instead incase I want to look at the original without edits
cf2.min_to_subway = cf2.min_to_subway.fillna(cf2.min_to_subway.mean(), axis=0) #Replacing empty values with the mean of the entire column
cf2.year_built = cf2.year_built.fillna(cf2.year_built.median(), axis=0) #Replacing empty values with the median of the entire column
cf2.loc[cf2['size_sqft']==0,'size_sqft'] = np.nan
cf2.size_sqft = cf2.size_sqft.fillna(cf2.size_sqft.mean(), axis=0)




In [232]:
print(train_df)

          addr_unit  building_id  bedrooms  ...  floornumber  line Occurrences
rental_id                                   ...                               
7236931        #22A       551248         4  ...          NaN   NaN         NaN
7331920        #406         3373         0  ...          4.0    06         NaN
7139224        #18A        70747         3  ...         18.0     A         NaN
7364827        #23E        42076         1  ...         23.0     E         NaN
7242151         #3B       319156         3  ...          3.0     B         NaN
...             ...          ...       ...  ...          ...   ...         ...
7411030        #33C        55852         2  ...         33.0     C         NaN
7397836         #D3        45853         1  ...          3.0     D         NaN
7154701        #20N        41170         0  ...         20.0     N         NaN
7259554          #3       121453         2  ...          3.0   NaN         NaN
7422694         #2E       130567         0  ...     

#### Build training data

In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 
    'size_sqft', 'no_fee', 'has_doorman', 'Occurrences'
]
#Year build and minute values are the only features with missing values
train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

#### Fit model

In [210]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

rf = RandomForestRegressor()
rf.fit(train_features, train_target)

ValueError: ignored

#### Predict and Measure Using Test 1

In [96]:
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = rf.predict(test_features)

mean_squared_error(test_df['rent'], test_df['predicted'])

2509440.2736219703

#### Combine Data, Predict Values for Test 2

In [105]:
master_df = train_df.append(test_df, sort=False)

master_df.loc[master_df['size_sqft']==0,'size_sqft'] = np.nan
master_df.size_sqft = master_df.size_sqft.fillna(master_df.size_sqft.mean(), axis=0)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

rf.fit(master_features, master_target)


gbr = GradientBoostingRegressor()
gbr.fit(train_features,train_target)
#gbr_pred = gbr.predict(train_features)
#mean_squared_error(train_target,gbr_pred)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [102]:
cf.drop(cf.tail(20).index,inplace=True)

print(cf['rent'].value_counts)

<bound method IndexOpsMixin.value_counts of 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1578   NaN
1579   NaN
1580   NaN
1581   NaN
1582   NaN
Name: rent, Length: 1583, dtype: float64>


#### Create Submission File for `test2`

In [106]:

submit1_features = cf[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
cf['predictions'] = gbr.predict(submit1_features)
cf['predictions'].to_csv('sample_submission1.csv', header=True)

cf['fake_rent'] = np.ones(cf['predictions'].shape) * master_target.median()
mean_squared_error(cf['predictions'], test_df['rent'])



10897528.46699751

In [110]:
cf2.drop(cf.tail(2).index,inplace=True)
submit1_df[feature_cols+['predictions']]

,bedrooms,year_built,bathrooms,min_to_subway,size_sqft,no_fee,has_doorman,Occurrences,predictions
0,0,1931.0,1.0,0.6000,450.0,0,0,11,2190.860013
1,1,2009.0,1.0,0.6167,667.0,1,1,11,3579.520323
2,3,1920.0,2.0,1.0667,2100.0,0,0,11,8416.898980
3,2,1930.0,1.0,0.6500,767.0,1,1,11,4213.948090
4,0,2005.0,1.0,3.9000,634.0,0,1,11,3160.800436
...,...,...,...,...,...,...,...,...,...
1578,1,1930.0,1.0,1.2000,925.0,1,0,6,2744.284029
1579,1,1927.0,1.0,0.8667,600.0,1,0,6,2095.162400
1580,1,1939.0,1.0,2.0667,680.0,0,0,12,2114.807052
1581,0,1956.0,1.0,14.3000,500.0,0,0,1,1485.207650


In [0]:
from google.colab import files

In [0]:
files.download("sample_submission1.csv")

#### Create Submission File for `test3`

In [112]:
submit2_features = cf2[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
cf2['predictions'] = gbr.predict(submit2_features)
cf2['']
cf2['predictions'].to_csv('sample_submission2.csv', header=True)

cf2['fake_rent'] = np.ones(cf2['predictions'].shape) * master_target.median()
mean_squared_error(cf2['predictions'], test_df['rent'])

11640704.686114011

In [113]:
test1 = submit2_df.sort_values(['predictions'], ascending=[1])
print(test1)

     addr_unit  building_id  bedrooms  ...  Occurrences   predictions fake_rent
1445      #712      1606783         0  ...            1   1485.207650    2800.0
622         #1       820096         0  ...            5   1537.288795    2800.0
634        #3A       950458         0  ...            5   1563.289762    2800.0
1205       #4J       130720         0  ...            5   1645.136690    2800.0
637       #4BB       951409         0  ...            5   1649.391684    2800.0
...        ...          ...       ...  ...          ...           ...       ...
213        #5C        63103         4  ...            3  17206.159024    2800.0
218    #13/14C        64603         5  ...            3  22800.979132    2800.0
1554      #542       119011         1  ...            1  29006.125386    2800.0
1420       #3D         5260         4  ...            2  33540.068301    2800.0
1362     #PH2C          313         4  ...            4  34775.030538    2800.0

[1583 rows x 41 columns]
